In [1]:
from langchain_community.document_loaders import HuggingFaceDatasetLoader #Permet de télecharger un dataset HuggingFace
from langchain_community.embeddings import HuggingFaceEmbeddings #On récupère un embedding adapté aux données HuggingFace
from langchain.indexes import VectorstoreIndexCreator # Permet à Langchain de transformer les données en vecteurs


In [2]:
datatset_name = "tweet_eval"
page_content_column = "text"
name = "stance_climate"

loader = HuggingFaceDatasetLoader(datatset_name,page_content_column, name)

In [4]:
embeddings = HuggingFaceEmbeddings()

index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])


C:\Users\Smart Data Building\AppData\Local\Temp\ipykernel_18076\4006776171.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


train-00000-of-00001.parquet:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.47k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/355 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/169 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40 [00:00<?, ? examples/s]

In [7]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.environ["HUGGINGFACEHUB_API_TOKEN"]


In [9]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    device=-1,
    pipeline_kwargs={"max_new_tokens":50},
    model_kwargs={"temperature":0}
)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\Smart Data Building\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Smart Data Building\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

c:\Users\Smart Data Building\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
c:\Users\Smart Data Building\anaconda3\Lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
def generate_answer(query):
    result = index.query(query, llm=hf)
    target_string = "Helpful Answer:"
    return result[result.find(target_string)+len(target_string):]

In [13]:
query = "Generate a tweet about climate change"

generate_answer(query)



"\n\nStep 2. To generate a Tweet about climate change, press F7.\n\nIn this example, I don't want the message to fall under the category of the comment that should be answered. The tweet can also take the form of"

In [14]:
query = "What is climate change"

generate_answer(query)

" As I mentioned, climate change is about human activity (usually humans). The human body's greenhouse gas emissions cause climate change. Humans and other non-human creatures cause climate change. In most cases, it isn't about human activity, but the human"